In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import sklearn
from collections import defaultdict

In [ ]:
from statsmodels.stats.multitest import fdrcorrection
import scipy.stats as stats

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import (KFold, train_test_split, cross_validate, RepeatedStratifiedKFold,
                                     cross_val_score, GroupKFold, StratifiedGroupKFold, StratifiedKFold)
from sklearn.metrics import (roc_auc_score, make_scorer, f1_score, confusion_matrix, roc_curve, accuracy_score, 
                             auc, precision_score)
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier

## Data Preprocessing

### Reading Data

In [ ]:
# initialize the file paths to hold all the images separated by sequence and SRS Date
root = '/shares/mimrtl/Users/Chengnan/SRS_Necrosis'
# initialize the file path to hold the extracted images
radiomics_path = os.path.join(root, 'Intact_Cohort/Radiomics')

In [ ]:
T1_batchfile = os.path.join(radiomics_path, 'T1_SkullStrip_BFC_IntStnd_nonorm_fbw_radiomic_features.csv')


In [ ]:
# Read T1Bravo data and add T1_ prefix to all the feature columns
extractedFeatures = pd.read_csv(T1_batchfile)
extractedFeatures = extractedFeatures.add_prefix('T1_')
extractedFeatures.rename(columns = {'T1_Mask': 'Mask'}, inplace = True)

# Get MRN_Date_lesion to specify a merge key for each lesion
extractedFeatures['MRN_Date_Lesion'] = extractedFeatures.apply(lambda row: (row['Mask'].split('/')[-3]
                                                               + '_' + row['Mask'].split('/')[-2]
                                                               + '_' + row['Mask'].split('/')[-1][:-12]).strip()
                                                               , axis=1).astype(str)
extractedFeatures

In [ ]:
# Get patient immunotherapy information
patientDetails = pd.read_excel(os.path.join(root, 'necrosis_project_sequences_intact.xlsx'))

# Get MRN_Date_lesion to specify a merge key for each lesion
patientDetails['MRN_Date_Lesion'] = patientDetails.apply(lambda row: (str(row['MRN']) if len(str(row['MRN'])) == 7 else '0' + str(row['MRN']))
                                                         + '_' + str(row['MRI_Date'])[:-9]
                                                         + '_' + row['RT_Location'].lower().strip()
                                                         , axis=1).astype(str)

# Replace the necrosis with label 0 and recurrence with label 1
patientDetails = patientDetails.replace('necrosis', 0)
patientDetails = patientDetails.replace('recurrence', 1)
patientDetails

In [ ]:
# Use MRN_Date_Lesion to merge radiomic_features with the clinical data
allFeatures = pd.merge(extractedFeatures, patientDetails, on='MRN_Date_Lesion', how = 'left')

# Make sure the data was merged correctly
with pd.option_context('display.max_rows', None):  # more options can be specified also
    display(allFeatures[['MRN_Date_Lesion', 'Necrosis_or_Recurrence']])

### Filtering Data

In [ ]:
# Keep only columns for radiomic features, labels, and subject MRN
keep = ['Necrosis_or_Recurrence', 'MRN']
remove_cols = [feature for feature in allFeatures.columns 
               if not (feature.startswith("T1_original") or feature in keep)]
filteredFeatures = allFeatures.drop(remove_cols, axis = 1)

# Remove rows with NA values, which signify missing label or features for that lesion
filteredFeatures = filteredFeatures.dropna()
filteredFeatures

In [ ]:
# Get training data and labels
X = filteredFeatures.drop(['MRN', 'Necrosis_or_Recurrence'], axis=1)
y = filteredFeatures['Necrosis_or_Recurrence'].astype(int)

# Get the MRN for each lesion, which is used later on to do patient-stratification when splitting into train and test
mrns = list(filteredFeatures.loc[:, 'MRN'])

# Strap into one variable
data = (X, y, mrns)

## Wilcoxon

In [ ]:
# Split dataset by label to do wilcoxon-signed rank test
recurrence = X.loc[y==1]
necrosis = X.loc[y==0]
features = necrosis.columns

# Run wilcoxon signed rank test on each of the features to see if statistically significant
p_values = []
for feature in features:
    res = stats.mannwhitneyu(necrosis[feature], recurrence[feature])
    p_values.append(res.pvalue)

# Correct p values for multiple testing
p_values_corrected = fdrcorrection(p_values, alpha=0.05, method='indep', is_sorted=False)[1]

# Count # of significant feats
count = 0
sig_features = []
for i in range(len(p_values)):
    print(features[i], p_values_corrected[i])
    if p_values_corrected[i] <= 0.05:
        sig_features.append((features[i], p_values_corrected[i]))
        count += 1
sig_features = sorted(sig_features, key=lambda x: x[1])
print("Total of {} significant feature(s) after fdr correction:\n {}".format(count, sig_features))

In [ ]:
recurrence['T1_original_glcm_InverseVariance']

## Box Plots

In [ ]:
# Select a feature
feature = 'T1_original_glcm_Id'
feature_name = 'glcm_Id'
# Creating box plots
plt.figure(figsize=(8, 6))
plt.boxplot([recurrence[feature], necrosis[feature]], labels=['Recurrence', 'Necrosis'])
plt.title('Box Plot of {} Across Both Cohorts'.format(feature_name))
plt.ylabel('{} Values'.format(feature_name))
plt.show()

In [ ]:
# Select a feature
feature = 'T1_original_glcm_InverseVariance'
feature_name = 'glcm_InverseVariance'
# Creating box plots
plt.figure(figsize=(8, 6))
plt.boxplot([recurrence[feature], necrosis[feature]], labels=['Recurrence', 'Necrosis'])
plt.title('Box Plot of {} Across Both Cohorts'.format(feature_name))
plt.ylabel('{} Values'.format(feature_name))
plt.show()

In [ ]:
# Select a feature
feature = 'T1_original_glcm_DifferenceAverage'
feature_name = 'glcm_DifferenceAverage'
# Creating box plots
plt.figure(figsize=(8, 6))
plt.boxplot([recurrence[feature], necrosis[feature]], labels=['Recurrence', 'Necrosis'])
plt.title('Box Plot of {} Across Both Cohorts'.format(feature_name))
plt.ylabel('{} Values'.format(feature_name))
plt.show()

## Running Models

### Defining CV Strategy

In [ ]:
from collections import Counter
from mrmr import mrmr_classif

In [ ]:
"""
CV based off StratifiedGroupKFold in sklearn, with additional functionality for repeats. 
Stratification splits the dataset such that each fold has a similar class balance as the original.
GroupKFold generartes folds by splitting across groups, such that each group doesn't appear both in the train and
test set of that fold. Additionally, each group appears exactly once in the test set across all folds.
"""
class RepeatedStratifiedGroupKFold():
    def __init__(self, n_splits=5, n_repeats=5, random_state=1):
        self.random_state = random_state
        self.n_repeats = n_repeats
        self.n_splits = n_splits

    def split(self, X, y, groups):
        tries = 0
        X = pd.DataFrame(X)
        y = pd.DataFrame(y)
        
        # For each repeat, generate a new KFold split through a new seed. 
        # Skip the split if any of folds have only one class in either the train or test set, as this causes
        # causes errors in AUC calculation.
        # If there are any invalid folds, generate another new KFold split.
        for idx in range(self.n_repeats):
            invalid = True
            while (invalid):
                cv = StratifiedGroupKFold(n_splits=self.n_splits, shuffle = True, random_state=self.random_state + tries)
                invalid = False
                tries = tries + 1
                # For each fold, test if train set or test set only has one class
                for train_index, test_index in cv.split(X, y, groups):
                    trainPosRatio = y.iloc[train_index].mean().item()
                    testPosRatio = y.iloc[test_index].mean().item()
                    if trainPosRatio == 0 or trainPosRatio == 1 or testPosRatio == 0 or testPosRatio == 1:
                        invalid = True
                        break
            
            # Yield StratifiedGroupKFold split for this repeat
            for train_index, test_index in cv.split(X, y, groups):
                yield train_index, test_index

    def get_n_splits(self, X, y, groups=None):
        # of splits = # of splits generated by StratiedGroupKFold * # of repeats
        cv = StratifiedGroupKFold(n_splits=self.n_splits, shuffle = True, random_state=self.random_state)
        return cv.get_n_splits(X, y, groups) * self.n_repeats

In [ ]:
# constants
N_SPLITS = 3
N_REPEATS = 20
rd=42

In [ ]:
# Test that the CV strategy generates splits with similar class balance to the original dataset, and that 
# the groups don't appear both in the train and test set of that fold
cv = RepeatedStratifiedGroupKFold(n_splits=N_SPLITS, n_repeats=N_REPEATS)
originalPosRatio = y.mean().item()

trainPosRatio = []
testPosRatio = []
for fold, (train_idxs, test_idxs) in enumerate(cv.split(X, y, mrns)):
    # split data
    mrns_train, mrns_test = np.array(mrns)[train_idxs], np.array(mrns)[test_idxs]
    if (set(mrns_train).intersection(mrns_test)):
        raise Exception("ERROR: Group appears in both train and test set of a fold. ")
    y_train, y_test = y.iloc[train_idxs], y.iloc[test_idxs]
    
    # get the class ratio for the train and test
    trainPosRatio.append(y.iloc[train_idxs].mean().item())
    testPosRatio.append(y.iloc[test_idxs].mean().item())

print("Original ratio for positive label: {}".format(originalPosRatio))
print("Ratio across train folds for pos label: {}+-{}".format(np.mean(trainPosRatio), np.std(trainPosRatio)))
print("Ratio across test folds for pos label: {}+-{}".format(np.mean(testPosRatio), np.std(testPosRatio)))

## Univariate Logit Analysis

In [ ]:
def cross_val_univariate(X, y, mrns, clf):
    cv = RepeatedStratifiedGroupKFold(n_splits=N_SPLITS, n_repeats=N_REPEATS, random_state=rd)
    
    train_scores = []
    test_scores = []
    specificity_scores = []
    sensitivity_scores = []
    f1_scores = []
    X = pd.DataFrame(X)
    y = pd.DataFrame(y)
    for fold, (train_idxs, test_idxs) in enumerate(cv.split(X, y, mrns)):
        # split data
        X_train, X_test = X.iloc[train_idxs], X.iloc[test_idxs]
        y_train, y_test = y.iloc[train_idxs], y.iloc[test_idxs]
                    
        # fit model
        clf.fit(X_train, y_train.values.ravel())
        
        # calculate scores
        train_auc_score = roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1])
        train_scores.append(train_auc_score)
        
        test_auc_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        test_scores.append(test_auc_score)
        
        tn, fp, fn, tp = confusion_matrix(y_test, clf.predict(X_test)).ravel()
        specificity = tn / (tn+fp)
        sensitivity = tp / (tp+fn)
        specificity_scores.append(specificity)
        sensitivity_scores.append(sensitivity)
        
        f1score = f1_score(y_test, clf.predict(X_test))
        f1_scores.append(f1score)
    return (train_scores, test_scores, specificity_scores, sensitivity_scores, f1_scores)


In [ ]:
# Univariate Analysis
lr = LogisticRegression(class_weight='balanced', solver = 'liblinear')
AUCforUniLogit = {}
for (featureName, featureData) in X.items():
    train_scores, test_scores, specificity_scores, sensitivity_scores, f1_scores = cross_val_univariate(featureData, y, mrns, lr)
    AUCforUniLogit[featureName] = [np.mean(test_scores), np.mean(specificity_scores), np.mean(sensitivity_scores), np.mean(f1_scores)]

# Sort top features by test AUC
AUCforUniLogitTop = dict(sorted(AUCforUniLogit.items(), key=lambda item: item[1][0], reverse=True))
AUCforUniLogitRank = list(AUCforUniLogitTop.keys())
AUCforUniLogitTop10 = dict(sorted(AUCforUniLogit.items(), key=lambda item: item[1][0], reverse=True)[:10])

# Display top features AUC, specifity, and sensitivity
print("{:<50} {:<15} {:<15} {:<15} {:<15}".format('feature','auc value','specificity', 'sensitivity', 'f1score'))
for key, value in AUCforUniLogitTop10.items():
    print("{:<50} {:<15.4f} {:<15.4f} {:<15.4f} {:<15.4f}".format(key, value[0], value[1], value[2], value[3]))

## Multivariate Analysis

In [ ]:
def select_features(X, y, fs):
    """
    Returns the set of features selected based on the given data and method.
    
    X: training features
    y: training labels
    fs: Feature selection method.
    """
    
    if fs == None:
        return X.columns
    elif fs[:4] == 'MRMR':
        k = int(fs[4:])
        features = mrmr_classif(X, y, k, show_progress=False)
        return features
    elif isinstance(fs, list):
        return fs
    else:
        raise Exception("Feature selection method provided ({}) isn't valid".format(fs))

In [ ]:
def plot_auc_curve(tprs, test_auc_scores):
    fig, ax = plt.subplots(figsize=(8, 8))
    plt.rcParams.update({'font.size': 12})
    # Plotting 0.5 AUC line
    ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
    
    # Plotting average roc auc curve across all folds
    fpr_points = np.linspace(0, 1, 100)
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    ax.plot(
        fpr_points,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (np.mean(test_auc_scores), np.std(test_auc_scores)),
        lw=2,
        alpha=0.8,
    )

    # Plotting +1 std and -1 std roc auc curves
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        fpr_points,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"Mean ROC curve with variability\n(Positive label 'recurrence')",
    )
    ax.axis("square")
    ax.legend(loc="lower right")
    plt.show()

In [ ]:
def executeExperiment(data, clfs, fs=None, normalize=True, resample=None, verbose=False, plot_roc=False):
    """
    data: Contains X, y, and mrns
    clfs: sklearn classifers to train model
    fs: Feature selection method to use
    normalize: Whether to normalize the features
    resample: Resampling strategy defined
    verbose: if true, will display results across each fold
    plot_roc: if true, the roc curve for each classifier will be plotted
    """
    # initialize variables
    (X, y, mrns) = data
    cv = RepeatedStratifiedGroupKFold(n_splits=N_SPLITS, n_repeats=N_REPEATS, random_state=rd)
    
    # For each classifier, define a dictionary to store results across all cross-validation folds
    results = []
    for i in range(len(clfs)):
        results.append(defaultdict(list))
    
    tprs = [[] for clf in clfs] # for roc curve
        
    # Keep track of features selected in each train-test split
    all_features = []
    
    for fold, (train_idxs, test_idxs) in enumerate(cv.split(X, y, mrns)):
        
        # split data
        X_train, X_test = X.iloc[train_idxs], X.iloc[test_idxs]
        y_train, y_test = y.iloc[train_idxs], y.iloc[test_idxs]
        
        # Perform feature selection within the training set, if necessary
        sel_feats = select_features(X_train, y_train, fs)
        # Keep track of the features selected across each fold
        all_features.extend(sel_feats)
        
        # Transform X_train and X_test based on selected features
        X_train = X_train[sel_feats]
        X_test = X_test[sel_feats]
        
        # Normalize features, necessary for linear models
        if normalize:
            orig_columns = X_train.columns
            sc = StandardScaler()
            
            # normalize X_train
            X_train = pd.DataFrame(sc.fit_transform(X_train))
            X_train.columns = orig_columns
            
            # normalize X_test
            X_test = pd.DataFrame(sc.transform(X_test))
            X_test.columns = orig_columns
        
        # resample if specified
        if resample:
            X_train, y_train = resample.fit_resample(X_train, y_train)
        
        # Apply classifiers to selected features
        for (i, clf) in enumerate(clfs):
            # fit model
            clf.fit(X_train, y_train)

            # calculate scores
            train_auc_score = roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1])
            test_auc_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
            tn, fp, fn, tp = confusion_matrix(y_test, clf.predict(X_test)).ravel()
            specificity = tn / (tn+fp)
            sensitivity = tp / (tp+fn)
            f1score = f1_score(y_test, clf.predict(X_test))
            accuracy = accuracy_score(y_test, clf.predict(X_test))
        
            # add to results dict
            results[i]['train_auc_scores'].append(train_auc_score)
            results[i]['test_auc_scores'].append(test_auc_score)
            results[i]['test_specificity_scores'].append(specificity)
            results[i]['test_sensitivity_scores'].append(sensitivity)
            results[i]['f1_scores'].append(f1score)
            results[i]['accuracy'].append(accuracy)
        
            # AUC graph
            fpr, tpr, _ = roc_curve(y_test, clf.predict_proba(X_test)[:, 1])
            interp_tpr = np.interp(np.linspace(0, 1, 100), fpr, tpr)
            interp_tpr[0] = 0.0
            tprs[i].append(interp_tpr)    

            if verbose:
                print("Results for classifier {} on Repeat {} and Fold {} ".format(clf, fold // N_SPLITS + 1, fold % N_SPLITS + 1))                
                print("Class balance for training (post resampling if any): ", Counter(y_train))
                print("Selected features: ", sel_feats)
                print("TRAIN SCORE: ", train_auc_score)
                print("TEST AUC SCORE: ", test_auc_score)
                print("TEST SPECIFITY: ", specificity)
                print("TEST SENSITIVITY: ", sensitivity)
                print()
    
    # averages across all fold
    print("RESULTS FOR ALL CLASSIFIERS USING FEATURE SELECTION METHOD: {}".format(fs))
    print("{:<30} {:<15} {:<15} {:<15} {:<15}".format('clf','auc value','specificity', 'sensitivity', 'f1score'))
    for (i, clf) in enumerate(clfs):
        (auc_mean, auc_std) = np.mean(results[i]['test_auc_scores']), np.std(results[i]['test_auc_scores'])
        (specificity_mean, specificity_std) = np.mean(results[i]['test_specificity_scores']), np.std(results[i]['test_specificity_scores'])
        (sensitivity_mean, sensitivity_std) = np.mean(results[i]['test_sensitivity_scores']), np.std(results[i]['test_sensitivity_scores'])
        (f1score_mean, f1score_std) = np.mean(results[i]['f1_scores']), np.std(results[i]['f1_scores'])
        (accuracy_mean, accuracy_std) = np.mean(results[i]['accuracy']), np.std(results[i]['accuracy'])
        
        print("{:<30} {:.4f}±{:<7.4f} {:.4f}±{:<7.4f} {:<.4f}±{:<7.4f} {:<.4f}±{:<7.4f} {:<.4f}±{:<7.4f}".format(
            clf.__class__.__name__, auc_mean, auc_std, specificity_mean, specificity_std, sensitivity_mean, sensitivity_std,
            f1score_mean, f1score_std, accuracy_mean, accuracy_std))
        if plot_roc:
            plot_auc_curve(tprs[i], results[i]['test_auc_scores'])
    
    # Get the top k overall features
    if fs:
        k = int(fs[4:])
        print("Top {} features across all folds: {}".format(5, Counter(all_features).most_common(5)))
    print("")


## Executing Experiments

In [ ]:
fs_methods = [None, 'MRMR50','MRMR25','MRMR10', 'MRMR5', 'MRMR3']
clfs = [RandomForestClassifier(n_estimators = 300, random_state=rd, class_weight='balanced', n_jobs=-1),
       LogisticRegression(penalty='l2', solver='liblinear', class_weight='balanced'),
       SVC(kernel='linear', probability=True, class_weight='balanced'),
       MLPClassifier(hidden_layer_sizes=(16, 16), random_state=rd, max_iter=1000),
       DummyClassifier(strategy="stratified", random_state=rd)]

In [ ]:
for fs in fs_methods:
    executeExperiment(data, clfs, fs=fs, verbose=False)